In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9354.30it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5242.39it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 2591651.03it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9654.46it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5546.36it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 61442.21it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 9297.98it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x1115c2f28>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x13ec386d8>,
 'vocab_size': 16674,
 'embedding_input_dim': 16674,
 'input_shapes': [(10,), (100,)]}

In [5]:
model = mz.models.DRMMTKS()

# load `input_shapes` and `embedding_input_dim` (vocab_size)
model.params.update(preprocessor.context)

model.params['task'] = ranking_task
model.params['mask_value'] = -1
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['top_k'] = 20
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 5
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'relu'
model.params['optimizer'] = 'adadelta'

model.build()
model.compile()

print(model.params)

model_class                   <class 'matchzoo.models.drmmtks.DRMMTKS'>
input_shapes                  [(10,), (100,)]
task                          Ranking Task
optimizer                     adadelta
with_embedding                True
embedding_input_dim           16674
embedding_output_dim          100
embedding_trainable           True
with_multi_layer_perceptron   True
mlp_num_units                 5
mlp_num_layers                1
mlp_num_fan_out               1
mlp_activation_func           relu
mask_value                    -1
top_k                         20


In [6]:
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1667400     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dot_1 (Dot)                     (None, 10, 100)      0           embedding[0][0]                  
          

In [7]:
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
test_x, test_y = test_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=test_x, y=test_y, batch_size=len(test_x))

In [12]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 408


In [13]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=4, use_multiprocessing=True)

Epoch 1/30
408/408 [==============================] - 7s 17ms/step - loss: 0.5841
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5988918016460031 - normalized_discounted_cumulative_gain@5(0.0): 0.6590526112135905 - mean_average_precision(0.0): 0.6149729212603963
Epoch 2/30
408/408 [==============================] - 16s 40ms/step - loss: 0.1286
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5986225503033814 - normalized_discounted_cumulative_gain@5(0.0): 0.6573151832954626 - mean_average_precision(0.0): 0.6081435794322055
Epoch 3/30
408/408 [==============================] - 17s 41ms/step - loss: 0.0236
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5770594699106932 - normalized_discounted_cumulative_gain@5(0.0): 0.642764019599169 - mean_average_precision(0.0): 0.5915165228479636
Epoch 4/30
408/408 [==============================] - 20s 48ms/step - loss: 0.0085
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5630076023551297 - normalize

In [ ]:
# append_params_to_readme(model)